In [ ]:
library(igraph)
library(repr)
source("func.r")

In [ ]:
N <- 1000
PARAM <-  2

# model <- "erdos"
# graph <- get_graph(model = model, only_bcc = T, add_spins = T, param1 = PARAM, layout_design = T)

graph = get_scalefree_with_constraint(N, PARAM, 2)
graph$layout_design <- layout_nicely(graph)
graph$spins <- generate_spins(vcount(graph))


vcount(graph)
# plot_graph(graph)
# bcc_size <- get_bcc_size(graph)
# cat("Size of the largest connected component:", bcc_size, "\n")


In [ ]:
plot_graph(graph)

In [ ]:
cat("Avg degree:", get_avg_degree(graph), "\n")
cat("Second moment:", get_moment_2(graph), "\n")
cat("Fourth moment:", get_moment_4(graph), "\n")
cat("TC:", get_Tc(graph), "\n")

In [ ]:
# use this: degree_distribution(graph)
node_degrees <- degree(graph)
xvalues <- (0 - 0.5):(max(node_degrees) + 0.5)

options(repr.plot.width = 16, repr.plot.height = 8)
par(mfrow = c(1, 2))
hist(node_degrees,
     breaks = xvalues,
     main = "Histogram of Integer Values",
     xlab = "Integers",
     ylab = "Counts",
     col = "blue",
     border = "black"
)
barplot(degree_distribution(graph)*N,
     names.arg = seq(0, length(degree_distribution(graph)) - 1),
     main = "Histogram of Integer Values",
     xlab = "Integers",
     ylab = "Frequency",
     col = "blue",
     border = "black"
)
lines(1:100, (1:100)^-PARAM)

In [ ]:
graph = simulate_ising(graph, kbt = 1.8, nstep = 50000)

In [ ]:
options(repr.plot.width = 28, repr.plot.height = 20)
par(mfrow = c(2, 3))
plot_graph(graph, set_size=F)
plot(graph$energies)
plot(abs(graph$magnetizations))
plot(graph$susceptibility)
# plot(graph$heat)


In [ ]:
paramsss = c(1)
for (i in 1:length(paramsss)) {
    N = 3000
    NSTEP <- 10000
    PERC <- 0.02

    # graph <- get_graph(model = model, only_bcc = T, add_spins = F, param1 = PARAM, layout_design = F)
    # graph <- delete_vertices(graph, which(degree(graph) == 1))
    # graph$layout_design <- layout_nicely(graph)
    # graph$spins <- generate_spins(vcount(graph))

    # model = "erdos"
    # PARAM = paramsss[i]
    # graph <- get_graph(model = model, only_bcc = T, add_spins = T, param1 = PARAM, layout_design = T)
    graph <- get_scalefree_with_constraint(N, PARAM, 2)
    graph$layout_design <- layout_nicely(graph)
    graph$spins <- generate_spins(vcount(graph))

    Tc <- get_Tc(graph)
    # kbts <- c(seq(Tc + 2, Tc + 0.35 + 0.1, -0.1), seq(Tc + 0.35, Tc - 0.35, -0.05), seq(Tc - 0.35 - 0.05, 0.01, -0.15))
    kbts <- c(seq(Tc + 0.50, Tc + 0.30 + 0.1, -0.1), seq(Tc + 0.3, Tc - 0.3, -0.05), seq(Tc - 0.3 - 0.15, max(Tc - 1, 0), -0.15))



    # kbts <- seq(0.1, 10, 0.33)
    cat("Ci sono ", length(kbts), " iterazioni di temperatura\n")

    cat("Inizio Procedura\n")
    graph <- multi_T_ising(NSTEP, kbts, graph, PERC)
    avgs_en <- graph$avgs_en
    avgs_m <- graph$avgs_m
    avgs_sus <- graph$avg_sus
    avgs_heat <- graph$avg_heat
    kbts <- graph$kbts

    formatted_datetime <- format(Sys.time(), format = "%Y_%m_%d_%H_%M_%S")
    filename <- paste("N", graph$nickname, PARAM, "_N", N, "_Step", NSTEP, "_", formatted_datetime, sep = "")
    filename

    save(graph, avgs_en, avgs_m, avgs_sus, avgs_heat, kbts, file = paste(filename, ".RData", sep = ""))
    # gc()
}


In [ ]:


# options(repr.plot.width = 14, repr.plot.height = 10)
# png(file = paste(filename, "_enT.png", sep = ""))
# plot(kbts, avgs_en)
# dev.off()

# options(repr.plot.width = 14, repr.plot.height = 10)
# png(file = paste(filename, "_mT.png", sep = ""))
# plot(kbts, avgs_m)
# dev.off()

# options(repr.plot.width = 14, repr.plot.height = 10)
# png(file = paste(filename, "_enall.png", sep = ""))
# plot(as.vector(t(all_energies)))
# dev.off()

# options(repr.plot.width = 14, repr.plot.height = 10)
# png(file = paste(filename, "_mall.png", sep = ""))
# plot(as.vector(t(all_magnets)))
# dev.off()

# options(repr.plot.width = 28, repr.plot.height = 10)
# par(mfrow = c(1, 2))
# plot(kbts, avgs_en)
# plot(kbts, abs(avgs_m))

# options(repr.plot.width = 28, repr.plot.height = 10)
# par(mfrow = c(1, 2))
# plot(as.vector(t(all_energies)))
# plot(as.vector(t(all_magnets)))


In [ ]:

Tc <- get_Tc(graph)
x <- seq(0, max(kbts), 0.001)
xc <- seq(Tc - 1, Tc + 1, 0.01)

title <- graph$name
# y <- xc
# if (graph$nickname == "sf") {
#     title <- paste(graph$name, "gamma:", graph$param, "m0:", graph$m0)
#     param <- graph$param
#     if (param > 2 && param < 3) {
#         y <- xc^(-1 / (3 - param))
#     } else if (param == 3) {
#         y <- exp(-2 * xc / get_avg_degree(graph))
#     } else if (param > 3 && param < 5) {
#         tau <- 1 - xc / Tc
#         y <- tau^(1 / (param - 3))
#     } else if (param == 5) {
#         tau <- 1 - xc / Tc
#         y <- (tau^(1 / 2)) / ((log(tau^-1))^(1 / 2)) # check if the ^-1 is the arg or the log
#     } else if (param > 5) {
#         tau <- 1 - xc / Tc
#         y <- tau^(1 / 2)
#     } else {
#         cat("Parametro non valido: ", param, " per il modello di Barabasi Albert (scalefree)\n")
        
#     }
# }

options(repr.plot.width = 10, repr.plot.height = 8)
plot(kbts, abs(graph$avgs_m)/N,
    main = title, xlab = "Kb * T", ylab = "Magnetization", col = "blue", pch = 19,
    # xlim = c(0, 4), ylim=c(0, 1)
)
# lines(xc, y, lwd = 2)
abline(v = Tc, col = "red", lwd = 2)
legend(
    # x = "topright", 
    x= 2.8, y = 1,
    legend = c("Computed Value", "Theoretical Critical Temperature Tc", "Expected Critical Behavior around Tc"),
    col = c("blue", "red", "black"),
    pch = c(19, NA, NA), lwd = c(0, 2, 2), lty = c(0, 1, 1)
)


In [ ]:
Tc <- get_Tc(graph)
x <- seq(0, max(kbts), 0.001)


options(repr.plot.width = 10, repr.plot.height = 8)
plot(kbts[2:length(kbts)], graph$avg_sus[2:length(kbts)],
    col = "blue", pch = 19,
    # xlim = c(0, 4), ylim=c(0, 1)
)
# lines(kbts, graph$, lwd = 2)
abline(v = Tc, col = "red", lwd = 2)
